In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
path = "/content/drive/My Drive/Khoa Luan/"

In [3]:
import pickle
import pandas as pd

In [ ]:
foody1_train = pd.read_csv(path+'NTC_SV/NTC_SV_train.csv').review.tolist()
foody1_train_label = pd.read_csv(path+'NTC_SV/NTC_SV_train.csv').label.tolist()
foody1_test = pd.read_csv(path+'NTC_SV/NTC_SV_test.csv').review.tolist()
foody1_test_label = pd.read_csv(path+'NTC_SV/NTC_SV_test.csv').label.tolist()

In [4]:
aivivn_reivew = pd.read_csv(path+'AIVIVN/preprocess_train.csv').review.tolist()
aivivn_label = pd.read_csv(path+'AIVIVN/preprocess_train.csv').label.tolist()

In [ ]:
aivivn = pd.DataFrame(list(zip(aivivn_reivew,aivivn_label)),columns=['review','label'])
aivivn.to_csv(path+'AIVIVN/preprocess_train.csv')

In [5]:
from sklearn.model_selection import train_test_split
aivivn_train,aivivn_test,label_aivivn_train,label_aivivn_test = train_test_split(aivivn_reivew,aivivn_label,test_size=0.3,random_state=48)

In [6]:
foody2_train = pd.read_csv(path+'data_foody.csv').review.tolist()
foody2_train_label = pd.read_csv(path+'data_foody.csv').label.tolist()
foody2_test = pd.read_csv(path+'test_foody.csv').review.tolist()
foody2_test_label = pd.read_csv(path+'test_foody.csv').label.tolist()

In [ ]:
vreview_review = foody1_train+aivivn_train+foody2_train
vreview_label = foody1_train_label + label_aivivn_train + foody2_train_label
test_vreview = foody1_test + aivivn_test + foody2_test
test_vreview_label = foody1_test_label + label_aivivn_test + foody2_test_label

In [ ]:
vreview = pd.DataFrame(list(zip(vreview_review,vreview_label)),columns=['review','label'])
vreview = vreview.dropna()
vreview_ = pd.DataFrame(list(zip(test_vreview,test_vreview_label)),columns=['review','label'])
vreview_ = vreview_.dropna()

In [11]:
data3 = aivivn_train + foody2_train
label_data3 = label_aivivn_train + foody2_train_label
test_data3 = aivivn_test + foody2_test
label_test_data3 = label_aivivn_test + foody2_test_label

In [ ]:
vreview.to_csv(path+'vreview_train.csv')
vreview_.to_csv(path+'vreview_test.csv')

In [13]:
data3_train = pd.DataFrame(list(zip(data3,label_data3)),columns=['review','label'])
data3_train = data3_train.dropna()
data3_test = pd.DataFrame(list(zip(test_data3,label_test_data3)),columns=['review','label'])
data3_test = data3_test.dropna()

In [15]:
data3_train.to_csv(path+'data3.csv')
data3_test.to_csv(path+'test3.csv')

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

In [17]:
classifiers = [
            # MultinomialNB(),
            # DecisionTreeClassifier(),
            # LogisticRegression(),
            # SGDClassifier(),
            LinearSVC(fit_intercept = True,multi_class='crammer_singer', C=1),
            #xgb.XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=15,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=27)
        ]

In [ ]:
for classifier in classifiers:
    steps = []
    steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),max_df=0.5, min_df=5)))
    steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
    steps.append(('classifier', classifier))
    clf = Pipeline(steps)
    clf.fit(data3_train.review.tolist(), data3_train.label.tolist())
    y_pred = clf.predict(data3_test.review.tolist())
    print(f1_score(Y_val, data3_test.label.tolist()))

In [20]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,data3_test.label.tolist()))

0.8555246422893482
